In [1]:
import csv, io
import json
from bs4 import BeautifulSoup
import codecs, prettytable
import requests
import codecs
from selenium.webdriver import Chrome


header_data={
		"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
		"Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
		"referer" : "https://forum.gamer.com.tw/B.php?bsn=60559&subbsn=7"
		}

In [2]:
# 取得所有題目的 snA
snA_list = []
for page in range(1, 19):
	r1 = requests.get(
			"https://forum.gamer.com.tw/B.php?bsn=60559&subbsn=7",
			# "https://cse.google.com/cse/element/v1?rsz=filtered_cse&num=10&hl=zh-TW&source=gcsc&gss=.tw&cselibv=57975621473fd078&cx=001994533065495648396:c3salsx3fty&q=iphone%20more%3Apagemap%3Athread-bsn%3A60559%20more%3Apagemap%3Athread-filename%3AC&safe=off&cse_tok=AJvRUv0MhpGmAsmqqz9xcdwefsvk:1589347127059&filter=1&sort=&exp=csqr,cc&callback=google.search.cse.api18276",
			# https://forum.gamer.com.tw/search.php?bsn=60559&q=iphone
			# &snA=48625
			headers=header_data,
			params={
				"page":str(page)			
				}
		)


	b1=BeautifulSoup(r1.text, "html.parser")
	a1 = b1.find_all("td",{"class":"b-list__summary"})


	for a2 in a1:
		snA = a2.find("a")
		# print(snA.attrs["name"])
		snA_list.append(snA.attrs["name"])
	
print(snA_list)
print(len(snA_list))
snA_list.remove("48476")


['49991', '49985', '48476', '49962', '49938', '49906', '49898', '49896', '49894', '49886', '49874', '49857', '49698', '49819', '49795', '49788', '49785', '49771', '49721', '49724', '49702', '49526', '49688', '49656', '49679', '49661', '49652', '49640', '49600', '49581', '49539', '49512', '49493', '49468', '49407', '49398', '49394', '49373', '49287', '49271', '49264', '49262', '49253', '49084', '49221', '49176', '49209', '49186', '49120', '49117', '49087', '49102', '49094', '49063', '49059', '48971', '48884', '48860', '48851', '48823', '48794', '48765', '48777', '48770', '48768', '48759', '48734', '48711', '48699', '48634', '48598', '48613', '48583', '48590', '48571', '48503', '48504', '48485', '48484', '48479', '48469', '48438', '48432', '48404', '48365', '48320', '48296', '48297', '48182', '48166', '48164', '48087', '48077', '46876', '42799', '47418', '46126', '47356', '47224', '47167', '47131', '46629', '46972', '46839', '46707', '46548', '46370', '46255', '46307', '46319', '46310', 

In [16]:
snA_list=[]
snA_list.append("49526")
print(snA_list)

['49526']


In [ ]:
import time

# 取得該snA展開留言編號

for snA in snA_list:
    r1 = requests.get(
                f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&subbsn=7",
                headers=header_data,
                params={
                    }
            )

    page_list=[]

    b1=BeautifulSoup(r1.text, "html.parser")
    floor = b1.find_all("p",{"class":"BH-pagebtnA"})
    

#     Commend_index = 0
    
    for f in floor:
        print(f.text[-1])
        page_list.append(f.text[-1])

    print(page_list)
    
    # 展開留言

    driver = Chrome("./chromedriver")
    for page in range(1, int(page_list[0]) + 1):
        
        r2 = requests.get(
                f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&page={page}&subbsn=7",
                headers=header_data,
                params={
                    }
            )
        
        
        print("處理第幾頁:", page)
        commentId_list=[]
        b2=BeautifulSoup(r2.text, "html.parser")
        a1 = b2.find_all("textarea")
        for a in a1:
#             print(a.attrs["data-snb"])
            commentId_list.append(a.attrs["data-snb"])
        print(commentId_list)
        
        # 打開網址
        driver.get(f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&page={page}&subbsn=7")
        # find -> find-element
        # find all -> find_elements
        for id_ in commentId_list:
            
            try:
                driver.find_element_by_id("showoldCommend_"+ id_).click()
                time.sleep(5)
                comment = driver.find_elements_by_class_name("reply-content__article")
    #             content = driver.find_elements_by_class_name("c-article__content")
                # content = driver.find_elements_by_("reply-content__article c-article")
                title = driver.find_elements_by_class_name("c-post__header__title ")
                # num = driver.find_element_by_css_selector("textarea.data-snb")
                # htmltext = driver.get_attribute('outerHTML') 
                with open("./output.csv", "a", encoding="utf-8", newline="") as file:
                    for til in (title):
                        writer = csv.writer(file)
                        writer.writerow(["0", til.text])
                        print(til.text)
                    for i, com in enumerate(comment):
                        writer = csv.writer(file)
                        writer.writerow([str(i+1), com.text])
                        print(com.text)                    
#             for con in content:
#                 print(con.text)
            except:
                comment = driver.find_elements_by_class_name("reply-content__article")
                title = driver.find_elements_by_class_name("c-post__header__title ")
                with open("./Apple_baha.csv", "a", encoding="utf-8", newline="") as file:
                    for til in (title):
                        writer = csv.writer(file)
                        writer.writerow(["0", til.text])
                        print(til.text)
                    for i, com in enumerate(comment):
                        writer = csv.writer(file)
                        writer.writerow([str(i+1), com.text])
                        print(com.text)
        

    driver.close() 
        

3
3
['3', '3']
處理第幾頁: 1
['148414', '148416', '148418', '148421', '148423', '148424', '148425', '148426', '148427', '148431', '148432', '148434', '148435', '148438', '148442', '148443', '148446', '148452']
【情報】iPhone SE (第二代) 規格整理
看到官網說電量跟iPhone 8相若，嗯，雖然便宜但先暫停一下
電量真的是買的關鍵....
這配置價錢就純在賣CPU順便清庫存了吧
買A13送破銅爛鐵
感謝整理
亮點真的就A13了...吧？
買A13送手機 ...
15K有無線充電有ax
還有防水
單純賣給需要4.7吋小手機的用戶吧，當初6S 4.7吋 128G台灣首發價32500元。 我是覺得這售價不是小手機用戶，大概沒吸引力。 畢竟XR 128G 19900。
我覺得目前無線充就是一個沒啥用的功能，充超慢又沒辦法連著充電盤一起拿著邊充邊玩，出遠門還要多帶一個盤子
無線慢主要是水果摳，三星都15W了
出門就帶旅充就好了幹嘛帶盤子
oppo無線充電都40w了
xr不下架就是故意讓人捏懶趴上11用的
捏懶趴高手
256G 若定價為19500,就可能有盤子買了。但可能是對XR 128G讓路。
雖然這價格也沒啥好要求的，但真的A13+LOGO大概就夠惹，我比較在意續航會不會比當時I8還差尤其用上A13後......
續航不會比較差吧？A13明顯比A11省電啊
326ppi....… 原本以為很香 結果臭掉了
??????
1萬買cpu ， 4500買周邊零件
. 官方給的答案是跟i8差不多，規格有寫
剛好把手上的i6換掉
優點：A13缺點：其他沒提到的部分（同價位對比）
缺點:鏡頭還是凸的
你不是小妹 鏡頭平的也可以，就是拍照更差而已。
優點就是A13了吧。。。720p的LCD硬說其實還好，但市面主流幾千塊的也都1080P了。
會買SE2的人還是會買不買的還是不買,信仰無價 等發售後一堆人開箱評測 口嫌體正直的人一堆
可能在某些果粉中 蘋果出這廉價品的,想必會影響他們高貴的蘋果,話說拿蘋果手機是在優越感屌甚麼!?
我只在乎電量，畢竟是蘋果
如果不在意螢幕、續航、超老外觀

猜對了，最多人買的128G會在17k上下(X
記憶體看國外寫是3GB，來源不是官網就是了
轉左的是買11,然後果黒也轉左？覺得混亂
為什麼不是使用I5S的殼,4.7吋全銀幕?? 不覺得很漂亮嗎?
我更想要4吋平鏡頭的i5s, se(一代)的殼，真的漂亮
安裝仔崩潰
請問整體大小跟哀鳳8ㄧ樣嗎 還是跟舊版SEㄧ樣 謝謝
結月葵我婆改成轉右了w
rockmanzerox大小和 iPhone 8 一樣
你不是小妹不行，這樣根本裝不下其他零件
樹上的Apple不能吃這才是果神樹
難怪上面會有果粉和果黑吵起來，原來是都轉到同個方向了 (X
飛刀之刃 蘋果樹上有毛毛蟲
完美手機 即將衝擊安卓廠
想知道電池行不行~~
這規格的價錢不知道會被拿什麼來比較
看官網續航應該不錯啦，這規格衝擊的與其說是安卓，不如說是X系列的二手市場
ちょろいね有些人會拿 Redmi K30 Pro, Honor 30 Pro 之類的來比較吧……
Nexium一切看拆解出來的電池容量， < 2000 我就收回『真香』這句話
那一定不香吧 4.7吋我沒記錯 以前的從沒超過2000mAh過
一切等 4 月 24 日就懂了
我還以為大小會跟一代一樣 但是內部跟 I11一樣
一代的機身根本塞不下 11 甚至是 8 的東西
樹上的Apple不能吃這樣才有意義阿 才有突破
對我來說SE的重點就是大小跟外觀像I5
只是i8的下架，至少確定蘋果正式進入產品線記憶體都有3G的時代，可能一兩年後i6s i7 i8會像兩年前的i6一樣越來越多遊戲不支援，最慘的應該就最近買i8的人了。
大小跟I8一樣，續航也說跟I8相同，GSM是標1821拉不知道會不會湊整數來個1900XD
delta 所有SE的文章、留言都看了；只有你有這想法、是不是還沒睡醒呀？
其實就是台8的換名升級機 跟SE沒什麼關係 甚至連背面玻璃都在 剩下只看RAM了 畢竟這台最多當副機遊戲機 電池可參考8 只能説後來的雙層主機板確實為電池空間做出貢獻（那麼代價是壽命
黑所以礙到你了嗎 看不順眼嗎!?
delta提醒你該醒了，不用在意！
我只關心電池，所以容量是多少？？
鐵蜘蛛 這大小+蘋果的尿性 不用太期待續行會有多好
感覺續航不會太好 但非重度使用的 應該是沒問題
不知道跟Xperia5、S10e這些安卓小手機比續航會怎麼樣。
delta 好可憐喔用巴哈看世界，

無線充電能延緩充電傳輸孔的耗損，還是不錯的!而且很多人無線充電盤是放在公司辦公桌上能隨放隨充。
重點應該是RAM吧, A11跟A13一般使用沒甚麼差別，會不會殺後台才是重點
猜對了，最多人買的128G會在17k上下(X
記憶體看國外寫是3GB，來源不是官網就是了
轉左的是買11,然後果黒也轉左？覺得混亂
為什麼不是使用I5S的殼,4.7吋全銀幕?? 不覺得很漂亮嗎?
我更想要4吋平鏡頭的i5s, se(一代)的殼，真的漂亮
安裝仔崩潰
請問整體大小跟哀鳳8ㄧ樣嗎 還是跟舊版SEㄧ樣 謝謝
結月葵我婆改成轉右了w
rockmanzerox大小和 iPhone 8 一樣
你不是小妹不行，這樣根本裝不下其他零件
樹上的Apple不能吃這才是果神樹
難怪上面會有果粉和果黑吵起來，原來是都轉到同個方向了 (X
飛刀之刃 蘋果樹上有毛毛蟲
完美手機 即將衝擊安卓廠
想知道電池行不行~~
這規格的價錢不知道會被拿什麼來比較
看官網續航應該不錯啦，這規格衝擊的與其說是安卓，不如說是X系列的二手市場
ちょろいね有些人會拿 Redmi K30 Pro, Honor 30 Pro 之類的來比較吧……
Nexium一切看拆解出來的電池容量， < 2000 我就收回『真香』這句話
那一定不香吧 4.7吋我沒記錯 以前的從沒超過2000mAh過
一切等 4 月 24 日就懂了
我還以為大小會跟一代一樣 但是內部跟 I11一樣
一代的機身根本塞不下 11 甚至是 8 的東西
樹上的Apple不能吃這樣才有意義阿 才有突破
對我來說SE的重點就是大小跟外觀像I5
只是i8的下架，至少確定蘋果正式進入產品線記憶體都有3G的時代，可能一兩年後i6s i7 i8會像兩年前的i6一樣越來越多遊戲不支援，最慘的應該就最近買i8的人了。
大小跟I8一樣，續航也說跟I8相同，GSM是標1821拉不知道會不會湊整數來個1900XD
delta 所有SE的文章、留言都看了；只有你有這想法、是不是還沒睡醒呀？
其實就是台8的換名升級機 跟SE沒什麼關係 甚至連背面玻璃都在 剩下只看RAM了 畢竟這台最多當副機遊戲機 電池可參考8 只能説後來的雙層主機板確實為電池空間做出貢獻（那麼代價是壽命
黑所以礙到你了嗎 看不順眼嗎!?
delta提醒你該醒了，不用在意！
我只關心電池，所以容量是多少？？
鐵蜘蛛 這大小+蘋果的尿

買iphone 11 pro max的錢我可以iphone ipad都有...真的要照相還是習慣帶單眼，手機只要拍人不會變猴子都還行
SE唯二的好只有比較輕和便宜⋯
XR是除了螢幕該有的都有，SE除了A13甚麼都沒有，根本沒有可比性吧
OO神XR 沒有 Wi-Fi 6，後置相機的人像模式只有三種
喜歡的是4.7的機身，但是屏佔比要高
SE 本來就很受歡迎吧....
不會 螢幕大小的市場區隔很強烈
我們這種數值狂是看性能，但是一般消費者只是看價格與螢幕大小
換一換，螢幕變大，結果價格變高，然後就在哪邊唱衰？有義意？
像我就是想用touch ID的客群，照你這樣改下去我就不買單了
沒玩遊戲現在手機效能已經過剩了...這種的其實挑螢幕還比較重要
玩遊戲用這種小螢幕，你會先近視，然後再老花
一台ipad mini 一台iphone才是工作休閒標配
他是說他躺下有182
蔬菜XD
強行歪樓..
大概是30M級巨人的手掌大小吧
原來是使用者定位的部分，配A13應該不是為了效能，而是為了配合以後的系統更新
蔬菜 不是 就是可以解鎖，faceid>>touch id
我身邊拿iPhone 用Face ID的自從戴口罩之後我常常看他們失敗，8成都會失敗，最後都是打密碼
枯草 真的se 讚
同樣用SE的來了，不過也不知道要不要換SE2，型不喜歡阿
我也是SE拿了很多年，倒是蠻想換這次SE2
國高中生這時候充滿社交 買蘋果的好處就是:IG拍攝限時動態上傳不會模糊
每次滑IG看到模糊的照片影片 100%都是安卓機 如果你是重度社交軟體使用者 用安卓PO過一次限時動態就受不了了 那畫質跟禎數被吃的很誇張
這也是我跳到蘋果的原因 身邊一堆安卓朋友上傳限時動態到現在還是模糊= =
【情報】iPhone SE (第二代) 規格整理
看到官網說電量跟iPhone 8相若，嗯，雖然便宜但先暫停一下
電量真的是買的關鍵....
這配置價錢就純在賣CPU順便清庫存了吧
買A13送破銅爛鐵
感謝整理
亮點真的就A13了...吧？
買A13送手機 ...
15K有無線充電有ax
還有防水
單純賣給需要4.7吋小手機的用戶吧，當初6S 4.7吋 128G台灣首發價32500元。 我是覺得這售價不是小手機用戶，大概沒吸引力。 畢竟XR 128G 19900。
我覺得目前無線充就是一個沒啥用的功能，充超慢又沒辦法連

哦，4.7吋的遊戲體驗
展想要小手機，又想要更新成最新的OS，可以正常使用APP的人，除了A13以外有得選嗎?
媽啦 4.7吋16：9看網頁都覺的痛苦 發神經嗎...
RoccatSE本來就不是要賣給想要大螢幕手機的人...,要大螢幕就是選XR,或是11,11 Pro Max
K2 5.2吋 18：9很大嗎？
Roccat不曉得你在說哪一款手機,XR跟11(6.1吋),11 Pro Max(6.5吋),iPhone沒有出過5.2吋螢幕大小的機種...
除了CPU以外要什麼沒什麼>> 請問樓主你出了多少錢買？(SE2020最大的貢獻就是讓所有Android中階機種大降價...)
K2 沒有就沒有啊..誰在跟你說iphone有沒有出過5.2機種
iPhone SE拉崩安卓中階手機！　全球4G手機準備跳樓清倉https://www.ettoday.net/news/20200417/1693670.htm
Roccat就說了「不曉得你在說哪一款手機」,把名字講出來聽聽,哪一款？！
你既然指名我問了，我也該知道你到底在問哪一支手機吧？
3a本來就在清貨底了 A51溢價太多 從開賣一路跳 跟se有什麼關係？
K2 說真的 沒興趣理你，不是你先take我嗎，不然我真的懶的回你
山中無歲月請去問各家新聞社記者,新聞不是我寫的!!
Roccat沒有要問問題就不要亂Tag指名問人什麼螢幕大還小,我tag你可是有針對你的話回覆的,覺得SE螢幕小的本來就該選別的大螢幕手機...(SE一開始客層就很明確的不是要給大螢幕需求的人)
K2 我又沒問你，take我幹嘛？吃飽太閒，找碴喔？
Roccat找碴的是你吧？！
K2 笑死....又一個邏輯掉到水溝的
記者要炒新聞 內文又自打臉 隨便立個標題騙點閱
標準的蘋果說大便很香，也會捧著大便說很香的人 退散
XR 早就做到人像模式了，只是慣例下放 SE 而已
樹上的Apple不能吃你沒搞清楚XR,新SE的人像模式的差異...,能夠即時手動自然調整淺景深程度，那是A13以後的機種才有支援的功能...
另外,"人像模式"是A10開始有的,"人像模式中的3D光影調整"是A11以後才有,"人像模式+更多種3D光影調整+背景虛化程度調整"是A13才有...,
有哪些軟體功能是看CPU決定的,不是看手機螢幕大小,當然也沒有什麼下放不下放的說法...
K2可是那

...

Jack 會買se的在未來幾年內也用不起5G
5G還要5.6年吧
到時候手機可能都壞了
飛刀之刃 主要是官方自己說 電池使用時間約與 iPhone8 相同 當然日常比較吃效能的東西 i8應該是噴比較快
weison 反正就等實測吧，沒那麼急，最久也是一週內就有答案，當然要直接拿官方說的跟i8相似也是妥當的，畢竟是唯一資訊。
應該省電很多 畢竟CPU是新製程
我也不會覺得會特別省電，因為也是有+很多新東西。 就是覺得他續航變長變短都有可能，還是等NDA解除後看實測吧。
XF+我果斷放生SE II了耶...買ROG I 快充，但SE I 繼續使用到5G起來(2 SIM)
棉姆咪雪兒5G中期大概就這幾年而已，六年後就要推6G了吧@@?
黃金忙內胡寶寶丶 4版務
電量如果連2開頭都沒有 那那...???A13 情何以堪 玩個吃雞一場差不多就要充電了 邊充邊玩 不會發熱降頻?...反正這隻跟打遊戲差遠了幹嘛配A13- -這城府很深A13不就是要用來打遊戲不然要幹嘛
看到中國電信商說ram3G電池跟i8一樣
吾乃墮天之王我暫時補上去，等拆解後再來更新
sora發熱降頻倒是不會，因為是單層主板，問題在於續航
只有A13不一樣 其他都跟i8一模一樣啊 去apple官網兩隻拉出來比較就知道了
唯一的差別就是多了A13本身附加的功能 還有 i8的正面跟背面顏色是有搭配的 SE2沒有 不管背面什麼顏色 正面都是黑的 這不是庫存 什麼是庫存
卓粉生777，價錢也跟當初差了10k，到底有啥好酸的zzz
黑怎麼賣都可以仇視蘋果啦 買便宜的這隻就說愛慕虛榮 買貴的其他隻就說盤子
壽司犬 還有人說買蘋果是為了裝b我快笑死，我自己拿rog2不就肥宅了（我本來就是ㄏㄏ）
黑就需求用途不一樣 還有外型外觀也都不一樣 就不懂這有什麼好酸 而且手機越賣越便宜對大家都是好事情
sora再熱也沒有a11熱，而且不可能打一場pubg就沒電==
我之前用75%壽命的i8都可以打3小時左右的pubg才沒電..
再說a13降頻還是比s855+強
你拿隔壁S系列CPU比啥東西 我的I6 打一場PUBG就 "差不多要充電" 這就是事實電量就是少誰跟你說 "打一場就沒電" 有人這樣說喔 你I8 3小時左右你是在騙誰啊 你確定是打PUBG??
我的I11 頂多打4場多一點 1場40分鐘 就快沒電了 你還在那邊吹牛- -厲害 我

樓上 但他的電量一點都不頂規啊..
沒玩遊戲還真的沒什麼差，4.7出門方便，在家用ipad玩遊戲比較高興
買iphone 11 pro max的錢我可以iphone ipad都有...真的要照相還是習慣帶單眼，手機只要拍人不會變猴子都還行
不可能，光4.7就不可能；前10也許有機會，但機會我認為不大。
4.7就很難向11或XR當成孝親機，不過大概會賣得還不錯，畢竟效能真的很強大，甚至比對面30K的S865都還強
XR除了螢幕發售前被質疑，實際發售後、好像沒被酸爆、反而是各種真香文。 硬體策略比5C好太多了。
黑 4.7完全不是問題..現在平板都快跟手機一樣普及了
SE這次我覺得不夠潮 2020了那上下真的太多
光是他那價錢就不知道可以霸榜多久了
XR 大暢銷是因為蘋果多次降低價格到一個合理的範圍內，要不然你以為會有很多人去買嗎?
4.7就是讚啦 黃金比例
4.7還是沒辦法單手操作吧！？
4.7 吋是可以單手操作
4.7跟5.8明明手機大小一樣⋯哪裡黃金比例，4.7全螢幕才是黃金比例吧
SE唯二的好只有比較輕和便宜⋯
XR是除了螢幕該有的都有，SE除了A13甚麼都沒有，根本沒有可比性吧
OO神XR 沒有 Wi-Fi 6，後置相機的人像模式只有三種
喜歡的是4.7的機身，但是屏佔比要高
SE 本來就很受歡迎吧....
想太多，這隻就是因為直接拿i8剩餘的庫存來賣，省去開發成本，才能用上A13，否則整個重新打掉+5.8吋螢幕絕對超過20K，而且會打到自家11 pro的市場
上面中肯，弄的太好、價格壓不下去；還打到自家的11 pro；一分錢一分貨。
不會 螢幕大小的市場區隔很強烈
我們這種數值狂是看性能，但是一般消費者只是看價格與螢幕大小
換一換，螢幕變大，結果價格變高，然後就在哪邊唱衰？有義意？
像我就是想用touch ID的客群，照你這樣改下去我就不買單了
沒玩遊戲現在手機效能已經過剩了...這種的其實挑螢幕還比較重要
玩遊戲用這種小螢幕，你會先近視，然後再老花
一台ipad mini 一台iphone才是工作休閒標配
你的手有182公分，真大
他是說他躺下有182
蔬菜XD
強行歪樓..
大概是30M級巨人的手掌大小吧
原來是使用者定位的部分，配A13應該不是為了效能，而是為了配合以後的系統更新
se2吧，便宜的高效能機，而且現在頻繁戴口罩，face id變不太實用，

CPU夠好拿來掛機就夠了 反正又不會盯著看 用內建腳本點就好
在什麼年齡用甚麼定位的產品這不意外吧
憨吉 電池1821mAh要用來掛機就得整天插著線了吧 ，這年代掛機不都用電腦模擬器了
鎖定4.7的用戶呀，要裝B不會買這台；跟XR差沒多少
當然在家插著線阿 我也不懂2020年了還有遊戲鎖模擬器 可以的話我也開模擬器 誰要看5~6吋的螢幕
要不是這台太慢出已經買了P4 我覺得有點大 不然..
話說以前小米大受歡迎，不就是CPU很強，其它都很差但是價格便宜
啊就有像我一樣喜歡小手機的人啊，要那麼看影片啥的我用平板還更爽，你手機做那麼大麻煩死了
A13再差，也比高通S865強
Neobodom沒人嘴SE用A13爛啊,相反的還是亮點
Neobodom 然而現在小米其他部分也不差了
看影片我有32"的電腦螢幕、55"的電視不用，為啥要靠6"以下的手機畫面?喜歡小手機的人，本來就不會特別喜歡拿手機搞太多事情。
沒有引領時代的瀏海 差評
Neobodom小米的CPU強，只存在於"安兔兔"這App上而已，現實上根本不是這麼一回事！(雷軍最聰明的就是推出了讓小米球員兼裁判的"安兔兔"這軟體,還一堆人相信它...)
你找一隻 ANDROID陣營 旗艦CPU 14K防水的給我看看
看到上面一堆護航的果粉拼命貶低小米安卓 真好笑
安卓cp值老大 s10 s20 其他品牌少笑死人了呵呵 安卓老大 世界第一 三星
Assassin 單要我找一個除了CPU外什麼都不好的安卓機，我就找不到了吧，這點真的只有蘋果敢做
Assassin前年的防水旗艦(X
A13那個功耗配這個電池 直接死去
A13遊戲順暢度肯定強的 只是那個電池只能掛尿袋吧
想省電掛機用電腦掛然後手機遠端就好了，反正都是要掛尿袋有差嗎
有老花可以做飛秒阿...
不喜歡手機搞那麼多事那要a13幹嘛 ？
那個電池真滴慘，這價位還是S10e是比較好的選擇。處理器比較差點（大部分人根本用不到那麼高的效能），但螢幕續航都屌打，相機也是。
處理器很好，電池跟螢幕…應該能吸引到一些想入坑的窮果粉
拿電影跟遊戲體驗說嘴的人很好笑吧，我家就有50吋大螢幕還跟你計較手機螢幕大小
神之愛龜幸運7 那你要a13幹嘛 ，滑fb.滑ig ? 不覺得很好笑嗎？
展 我有說我要a13嗎？單純覺得看電影玩遊戲都在手機上還拿來說嘴很可笑
喔 原來手遊不能在手機上玩啊？


ELLIS 一個舉例而已 阿正常一般社交軟體使用者 的確會把一天1/3時間花在Ig阿== 又不是講其他人
ELLIS我也不會特地去遊戲手機串下面講說：講的遊戲就是人生一樣==
壽司犬 一般正常人不是會花三分一時間來工作嗎？哈哈
無敵小學雞你把我的三分之二忽略? 我是說用一天用手機的時間
跟SE1一樣，就是賣給小手機用戶的；不會年年都出；估計下次的模板，就是XS了
太棒了
蘋果出4.7吋=重新定義~安卓出4.7吋=電子 X X (總結)信仰=無價
雷公 我相信安卓如果出一支4.7吋S865/8/256GB+1080P全面屏，廣角+超廣角雙鏡頭+2500mAh+雙喇叭+線性馬達+IP67+屏下指紋，賣20000內，很多人應該還是會真香的
蔬菜 2500mAh一聽就很臭
☭粉紅犬ㄐㄐ☭ 4.7吋全面屏的話 大概長寬11/6.5公分，要在如此細小的機身放入2500以上的電池，並不是非常容易的事，請你妥善考慮
雷公 你也知道信仰無價 怎不想想蘋果怎麼營造這些信仰的 營造到台灣月銷售+市佔率第一 這是不爭的事實 別崩潰了
雷公 安卓也可以營造信仰阿 你快去幫忙建議營造安卓信仰的方法 在這邊戰果粉不會讓蘋果市佔變第二
台灣市佔？呵呵你眼裡的世界只有台灣？
雷公 我待在台灣 看台灣的購買趨勢錯了嗎？ 那如果單看全球月銷售量也確實是蘋果霸佔好幾名欸...
雷公 我只知道，你管真多；整天管別人怎麼用手機。
acclamator 我也有跟他借來把玩下，經過z3 z5那代之後其實發熱對我來說都是可接受的範圍
acclamator 然後sony有點尷尬的是他如果電池要再大，機身勢必還要更大然後就會變成有點過長
發高燒流鼻水 你可以看一些sony的手機拆解影片 他們都會提到 sony的設計思路與別人不一樣 也是因為sony的堅持才會讓電量不行
發高燒流鼻水 另外 別的廠商都能做到高集成度 就sony會在旗艦機內留下空置 如果這些改善可能銷量就超越中國機了
acclamator 索尼主要的問題是因為主板做成了L型先壓迫了電池的空間然後為了不讓電池膨脹破主板又加厚邊框導致電池再被擠迫
我覺得 Localytics 的資料有點老舊，可以找 2019 的會比較好
小螢幕方便多了，隨便褲子口袋一塞就可以出門了，以前看到同事買plus結果他只能用手拿，頓時聯想到以前的黑金剛手機
這300年前的資料一點參考

每個人需求本來就不同....你喜歡你的就好了 別人喜歡的你又何必否定
真·撿角三人組 組長 網路買手機，我覺得不是每個人都會這麼做；除了官方或連鎖平台。
台灣每個月銷售量最多也是蘋果 用巴哈跟PTT看台灣也不準ㄏㄏ
用銷量就不用來討論區了 在家看報紙看那隻賣最好 買那個就可以了
售價就是首要考量的點了 或許其他效能都輸一點 但是現實就是預算只能買非旗艦的特賣機
淨殘念 銷量好，市場支援度就高呀，不管是維修、備料、配件週邊。 賣不好的，只有超便宜花樣殼能選。
與其說反指標，倒不如說會在這討論的人自然是對手機配備有一定程度了解的CP廚了，跟CP廚的數量比起來，喜歡蘋果Logo的人可是要多太多
11 pro也14.4cm，影片體驗更好
太貴阿，我的使用需求不用買這麼好的。在外也只聽音樂不看影片
除非不要前置鏡頭⋯不然技術不到，只能取捨
挖孔水滴還比較能接受，前攝像頭就算不自拍還是會微量用到，不過真心覺得上下留邊不是問題
因為要塞Face Id模組吧，所以瀏海才會特別大⋯
以前我也不接受呀，用了發現蠻香！ 比沒劉海的IPHONE操作上，方便不止一個次元！
使用過的智能手機：蘋果3 > 蘋果4 > 三星S3 > 蘋果6 > 蘋果XR(服役中)
説真的，還是蘋果好用，不是信仰而已
你這個捏也捏太大力了吧
出社會之後，買車買房都幾十萬幾百萬了，8000也是錢但真的不算捏
8000摳，少喝幾杯手搖杯或幾次雞排就有了
8000不算捏了吧...
捏到蛋蛋爛掉
我國中時期拿nokia滑蓋，引領風潮
很多人都是買logo而已啦，然後網路上放自拍覺得自己很屌
我是買來玩遊戲，舊機六還能玩一部分的遊戲。
Whistle 玩遊戲別用這麼小的，XR好選擇
真·撿角三人組 組長 我是去年換小八的，目前沒打算要換，螢幕小脖子的確有點痠。
真·撿角三人組 組長 那買平板也是？ 不要因為安卓平板太差就認為ipad的CP值高、買了合理；手機你不喜歡就是為了潮。 要抵制也堅持點。
真的 ig對安卓的優化像屎一樣 之前用安卓直接拍真的有夠糊
空氣感我愛騎爆改勁戰 一堆人拍照推安卓 結果拍的美美的 想上傳IG限時動態分享給朋友看卻是一團糊
這點iPhone 大贏安卓，但這不是安卓的錯，是ig的垃圾優化
還好我ig從來只是看妹子用
還有 Wi-Fi 6、人像模式、人像光線啊，不過說真的也只有這些而已
ELLIS 一

In [10]:
# 展開留言
import time
driver = Chrome("./chromedriver")
# 打開網址
driver.get("https://forum.gamer.com.tw/C.php?bsn=60559&snA=49661&tnum=1&subbsn=7")
# find -> find-element
# find all -> find_elements

# driver.find_element_by_id("showoldCommend_149528").click()
# time.sleep(3)
comment = driver.find_elements_by_class_name("reply-content__article")
content = driver.find_elements_by_class_name("c-article__content")
# content = driver.find_elements_by_("reply-content__article c-article")
# num = driver.find_element_by_css_selector("textarea.data-snb")
# htmltext = driver.get_attribute('outerHTML') 


for con in content:
	print(con.text)

for com in comment:
	print(com.text)

我有試過itunes，但是根本沒有辦法傳任何東西
icloud又非常慢，傳個小影片電腦好了手機的icloud資料夾要很久之後才會顯示
請問有什麼好方法嗎？
推薦一個軟體 sendanywhere，我家開照相館客人傳送照片都用這個方式，而且不限於照片，影片音訊文件都可以
手機跟電腦都安裝究OK了
既然都用蘋果了，你也只能麻煩一點的慢慢傳了，要直接把手機當隨身碟使用，無解。
感謝大家的建議，我每個都試試看好了
正解:購買一隻S10
蔬菜下一隻手機是有想換回安卓了
很棒的app
